## Load testing scenarios.

- Create multiple users with individual API keys
- Random mapping of users to meetings
- Bot lifecycle management
- Background monitoring with timestamps
- Pandas integration for data analysis


In [1]:
!pip install pandas
!pip install vexa-client

In [2]:
# Import required libraries
import os
import sys
import pandas as pd
from IPython.display import clear_output, display
import time

# Add the new_tests directory to the path
sys.path.append('../testing')

# Import our test library
from test_suite import TestSuite
from vexa_client.vexa import parse_url


ModuleNotFoundError: No module named 'test_suite'

In [3]:
# Configuration
BASE_URL = "http://localhost:18056"  # Change to your Vexa API URL
ADMIN_API_KEY = 'token'  # Set this environment variable

# Test parameters
NUM_USERS = 2
MEETING_URLS = [
    "https://meet.google.com/ubx-sfgu-tau",
#    "https://meet.google.com/nbk-dfdu-xkj"
]



In [4]:
# Initialize TestSuite
test_suite = TestSuite(
    base_url=BASE_URL,
    admin_api_key=ADMIN_API_KEY,
    poll_interval=2.0 
)

print("TestSuite initialized")


TestSuite initialized


In [5]:
# Step 1: Create users
test_suite.cleanup()
users = test_suite.create_users(NUM_USERS)
print(f"Created {len(users)} users")
# Step 2: Create random user-meeting mapping
mapping = test_suite.create_random_mapping(MEETING_URLS)
bots = test_suite.create_bots(bot_name_prefix="DemoBot")
results = test_suite.start_all_bots(language='en', task='transcribe')
test_suite.start_monitoring()

Cleaning up TestSuite...
Monitoring not running
TestSuite cleanup completed
Creating 2 users...

DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': 'test_user_0_4742@example.com', 'name': 'Test User 0', 'max_concurrent_bots': 2}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Thu, 25 Sep 2025 13:53:29 GMT, Thu, 25 Sep 2025 13:53:29 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '162', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"test_user_0_4742@example.com","name":"Test User 0","image_url":null,"max_concurrent_bots":2,"data":{},"id":26,"created_at":"2025-09-25T13:53:30.399737"}...

DEBUG: Making POST request to http://localhost:18056/admin/users/26/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response stat

DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Thu, 25 Sep 2025 13:53:30 GMT, Thu, 25 Sep 2025 13:53:30 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '397', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":19,"user_id":26,"platform":"google_meet","native_meeting_id":"ubx-sfgu-tau","constructed_meeting_url":"https://meet.google.com/ubx-sfgu-tau","status":"requested","bot_container_id":"c789a62c6c96b06ebde83bcbd087423facfcd0244592e3d106be0634d83f5ac6","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-25T13:53:30.560911","updated_at":"2025-09-25T13:53:30.584127"}]}...

DEBUG: Making GET request to http://localhost:18056/transcripts/google_meet/ubx-sfgu-tau
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'si91awRGGIS3Ui1ySM0646QeApIfeOyijEjCMeAq'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Thu, 25 Sep 2025 13:53:30 GMT, Thu, 25 Sep 2025 13:

In [150]:
pd.options.display.max_columns = None
now = pd.Timestamp.now()

df = test_suite.get_latest_dataframe()
print(df['transcription_latency'].mean())
df[['bot_id','platform','meeting_status','created_at','transcription_latency','current_status','detected_languages','segments_count']].sort_values(by='transcription_latency', ascending=False)

6.128147


,bot_id,platform,meeting_status,created_at,transcription_latency,current_status,detected_languages,segments_count
0,DemoBot_0,google_meet,joining,None,7.556806,joining,[en],13
1,DemoBot_1,google_meet,active,None,4.699488,active,[en],17


In [ ]:
bot

In [151]:
bot = bots[0]
bot.user_client.get_meetings()


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'si91awRGGIS3Ui1ySM0646QeApIfeOyijEjCMeAq'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Thu, 25 Sep 2025 13:55:53 GMT, Thu, 25 Sep 2025 13:55:53 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '517', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":19,"user_id":26,"platform":"google_meet","native_meeting_id":"ubx-sfgu-tau","constructed_meeting_url":"https://meet.google.com/ubx-sfgu-tau","status":"joining","bot_container_id":"c789a62c6c96b06ebde83bcbd087423facfcd0244592e3d106be0634d83f5ac6","start_time":null,"end_time":null,"data":{"status_transition":[{"to":"joining","from":"requested","source":"bot_callback","timestamp":"2025-09-25T13:53:35.044494"}]},"created_at":"2025-09-25T13:53:30.560911","updated_at":"2025-09-25T13...


[{'id': 19,
  'user_id': 26,
  'platform': 'google_meet',
  'native_meeting_id': 'ubx-sfgu-tau',
  'constructed_meeting_url': 'https://meet.google.com/ubx-sfgu-tau',
  'status': 'joining',
  'bot_container_id': 'c789a62c6c96b06ebde83bcbd087423facfcd0244592e3d106be0634d83f5ac6',
  'start_time': None,
  'end_time': None,
  'data': {'status_transition': [{'to': 'joining',
     'from': 'requested',
     'source': 'bot_callback',
     'timestamp': '2025-09-25T13:53:35.044494'}]},
  'created_at': '2025-09-25T13:53:30.560911',
  'updated_at': '2025-09-25T13:53:35.038596'}]

In [147]:
[s['text'] for s in bot.get_transcript()['segments'][-5:]]


DEBUG: Making GET request to http://localhost:18056/transcripts/google_meet/ubx-sfgu-tau
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'si91awRGGIS3Ui1ySM0646QeApIfeOyijEjCMeAq'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Thu, 25 Sep 2025 13:55:40 GMT, Thu, 25 Sep 2025 13:55:40 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '3621', 'content-type': 'application/json'}
DEBUG: Response content: {"id":19,"platform":"google_meet","native_meeting_id":"ubx-sfgu-tau","constructed_meeting_url":"https://meet.google.com/ubx-sfgu-tau","status":"joining","start_time":null,"end_time":null,"segments":[{"start":0.0,"end":5.4,"text":" I don't know what's going on guys. Let's let's see we've got the locks","language":"en","created_at":null,"speaker":"Dmitriy Grankin","absolute_start_time":"2025-09-25T13:54:15.477207+00:00","absolute_end_time":"2025-09-25T13:54:20.877207+00:00"},{"start":5.4,"end":20....


[" what we've got here, we've got it active and we've got the transcript and we'll see the last",
 ' with the last five',
 " and we'll see the last five and the latency here looks good to me",
 ' yeah latency is okay what about this one what about this one and this one works',
 ' too']

In [2]:
bot.get_transcript()

NameError: name 'bot' is not defined

In [152]:
test_suite.cleanup()

Cleaning up TestSuite...
Stopping background monitoring...
Background monitoring stopped
Stopping 2 bots...

DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/ubx-sfgu-tau
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'si91awRGGIS3Ui1ySM0646QeApIfeOyijEjCMeAq'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Thu, 25 Sep 2025 13:57:26 GMT, Thu, 25 Sep 2025 13:57:27 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...
Stopped bot DemoBot_0

DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/ubx-sfgu-tau
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'JIsrneuwmQQjdylc2M5UvifOCLTuGQ532EUNSODv'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Thu, 25 Sep 2025 13:57:26